---
# Lambda School Data Science - Intro to Pandas
---
# 05 - Cleaning Data
---



##STOP! BEFORE GOING ANY FURTHER...  


1. Click "File" at the top.
2. Then, "Save a Copy in Drive."
3. Change the file name to "FIRSTNAME_LASTNAME_1_2"  

Now you have a copy of this notebook in your Drive account. This is the copy you'll edit and submit. Be sure to do this for ***every*** assignment!



### Import pandas

In [0]:
import pandas as pd

### Import the data in the CSV file
You can find the data [here](https://raw.githubusercontent.com/axrd/datasets/master/gdpmessy2.csv). Should have a shape of (222,3). Use this [resource](https://www.iban.com/country-codes) to clean values correctly.

In [0]:
Dir = "https://raw.githubusercontent.com/axrd/datasets/master/gdpmessy2.csv"
Data = pd.read_csv(Dir)
print("Original sheet: %d rows, %d columns" % Data.shape)
# Fix column names:
Data.columns = ["INDEX TO BE DELETED", "COUNTRY", "GDP [$B]", "CODE"]

Original sheet: 222 rows, 4 columns


### Quickly inspect the head

In [0]:
Data.head(10)

,INDEX TO BE DELETED,COUNTRY,GDP [$B],CODE
0,0,Afghanistan,21.71,AFG
1,1,Albania,13.40,ALB
2,2,Algeria,227.80,DZA
3,3,American Samoa,0.75,ASM
4,4,Andorra,4.80,AND
5,5,Angola,131.40,AGO
6,6,NaN,0.18,AIA
7,7,Antigua and Barbuda,1.24,ATG
8,8,Argentina,536.20,ARG
9,9,Armenia,10.88,ARM


### Inspect the tail

In [0]:
Data.tail(10)

,INDEX TO BE DELETED,COUNTRY,GDP [$B],CODE
212,212,Uruguay,55.60,URY
213,213,Uzbekistan,63.08,UZB
214,214,Vanuatu,0.82,VUT
215,215,Venezuela,209.20,VEN
216,216,Vietnam,187.80,VNM
217,217,U.S. Virgin Islands,5.08,VIR
218,218,West Bank,6.64,WBG
219,219,Yemen,45.45,YEM
220,220,Zambia,25.61,ZMB
221,221,Zimbabwe,13.74,ZMB


### What is the shape of the DataFrame? Does it correspond to what we expected? 

In [0]:
print("Nope, it doesn't. There is %d spare column." % (Data.shape[1] - 3))

Nope, it doesn't. There is 1 spare column.


### Do you need to change the index?

In [0]:
###### Yes, we need to delete the index column from source:
Data.drop("INDEX TO BE DELETED", axis = 1, inplace = True)
###### Then:
print("Index changed: %d rows, %d columns" % Data.shape)

Index changed: 222 rows, 3 columns


### Missing values?
Find all the missing values and correct them. 

In [119]:
# Step 1) Find missing values and store in "Row1":
print("Rows with missing values:\n------------------------")
Row1 = []
for Column in Data.columns:
  print("* In %8s column:" % Column)
  Dnan = Data[Data[Column].isnull()] # Get cells with NaNs in column.
  Row1.append(Dnan.index.tolist())   # Add their row numbers to error list.
  print(Dnan)

Rows with missing values:
------------------------
* In  COUNTRY column:
    COUNTRY  GDP [$B] CODE
6       NaN      0.18  AIA
20      NaN      1.67  BLZ
106     NaN      0.16  KIR
206     NaN      0.04  TUV
* In GDP [$B] column:
       COUNTRY  GDP [$B] CODE
16  Bangladesh       NaN  BGD
70      France       NaN  FRA
96     Ireland       NaN  IRL
* In     CODE column:
          COUNTRY  GDP [$B] CODE
14   Bahamas, The      8.65  NaN
55        Denmark    347.20  NaN
90        Hungary    129.70  NaN
100       Jamaica     13.92  NaN
200          Togo      4.84  NaN


### Find duplicate values  
Clean where necessary:  
*   France has a GDP of 2902.0 billion.
*   Ireland has a GDP of 245.80 billion.
*   Bangladesh has a GDP of 186.60 billion.

In [0]:
# Step 2) Complete missing values:
Missing = {"COUNTRY"  : ["Anguilla", "Belize", "Kiribati", "Tuvalu"],
           "GDP [$B]" : [2902.0, 245.8, 186.6],
           "CODE"     : ["BHS", "DNK", "HUN", "JAM", "TGO"] }

for n_col, Column in enumerate(Data.columns):
  for n_chg, Change in enumerate(Missing[Column]):
    Data.at[Row1[n_col][n_chg], Column] = Change

In [0]:
# Step 3) Find duplicate values and store in "Row2":
print("Rows with duplicate values:\n--------------------------")
Row2 = []
for Column in Data.columns[Data.columns != "GDP [$B]"]: # GDP dups irrelevant.
  print("* In %8s column:" % Column)
  Dnan = Data[Data[Column].duplicated(keep = False)] # Cells with duplicates.
  Row2.append(Dnan.index.tolist())     # Add their row numbers to error list.
  print(Dnan)

Rows with duplicate values:
--------------------------
* In  COUNTRY column:
Empty DataFrame
Columns: [COUNTRY, GDP [$B], CODE]
Index: []
* In     CODE column:
                  COUNTRY  GDP [$B] CODE
8               Argentina    536.20  ARG
209  United Arab Emirates    416.40  ARG
220                Zambia     25.61  ZMB
221              Zimbabwe     13.74  ZMB


In [0]:
# Step 4) Replace incorrectly duplicated values:
Replace = {"COUNTRY"  : [],
           "GDP [$B]" : [],
           "CODE"     : [[209, "ARE"], [221, "ZWE"]] }

for n_col, Column in enumerate(Data.columns):
  for Change in Replace[Column]:
    Data.at[Change[0], Column] = Change[1]

### Verify the change persisted!

In [0]:
Changed = []
for column in Row1:
  for row in column:
    Changed.append(row)
for column in Row2:
  for row in column:
    Changed.append(row)
print("Verified! These rows have been fixed:")
Data.iloc[sorted(Changed), :]

Verified! These rows have been fixed:


,COUNTRY,GDP [$B],CODE
6,Anguilla,0.18,AIA
8,Argentina,536.20,ARG
14,"Bahamas, The",8.65,BHS
16,Bangladesh,2902.00,BGD
20,Belize,1.67,BLZ
55,Denmark,347.20,DNK
70,France,245.80,FRA
90,Hungary,129.70,HUN
96,Ireland,186.60,IRL
100,Jamaica,13.92,JAM


### Submit your assignment notebook! (Make sure you've changed the name to FIRSTNAME_LASTNAME_1_2): 

1.  Click the Share button in the upper-right hand corner of the notebook.
2.  Get the shareable link.
3.  Set condition to: "Anyone with the link can comment."
4.  TBD 


---
